In [ ]:
! pip install Flask flask-ngrok
!pip install fastapi nest-asyncio pyngrok uvicorn

     |████████████████████████████████| 51kB 2.7MB/s 
     |████████████████████████████████| 747kB 7.1MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 
     |████████████████████████████████| 10.1MB 25.5MB/s 
     |████████████████████████████████| 61kB 7.0MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.0.5-cp37-none-any.whl size=19246 sha256=20cbf979c09980b98dc516636443d6b17738d8b8ee5a589cb38a2c23c6bbacac
  Stored in directory: /root/.cache/pip/wheels/0c/13/64/5ebbcc22eaf53fdf5766b397c1fb17c83f5775fdccf0ea1b88
Successfully built pyngrok


In [ ]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2021-04-29 19:23:05--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-04-29 19:23:06--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1608 (1.6K) [text/plain]
Saving to: ‘STDOUT’

setup Colab for PySpark 3.0.2 and Spark NLP 3.0.2
-                   100%[===================>]   1.57K  --.-KB/s    in 0s      

2021-04-29 19:23:06 (44.3

In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn

In [ ]:

import sparknlp
from sparknlp.annotator import *
from sparknlp.base import *

import pandas as pd
import pickle
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml.feature import HashingTF, IDF, StringIndexer, SQLTransformer,IndexToString

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression


from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.functions import col
from pyspark.sql import SQLContext
from pyspark.ml.evaluation import BinaryClassificationEvaluator 

import re
from pyspark.sql import types as t
from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify, request
from pydantic import BaseModel
from fastapi import Request
from fastapi import Request, FastAPI

import nest_asyncio
from pyngrok import ngrok
import uvicorn

#import gcsfs


In [ ]:

def get_replacements(html_tags_open):
    html_tags_close=[]
    for tag in html_tags_open:
        html_tags_close.append(tag[0:1]+'/'+tag[1::])
    replacements = [
    ('<code>.*?</code>',' code shown here '),
    ('http\S+',''),
    ('<a href.*?>',''),
    ('<img src.*?>', ''),
    ('<pre class.*?>', ''),
    ('\n', ''),
    ('$',''),
    ('-+','')]
    for tag_open,tag_close in zip(html_tags_open,html_tags_close):
        replacements.append((tag_open,''))
        replacements.append((tag_close,''))
    return replacements
    


def cleaning_response(item):
    
    html_tags_open=['<a>','<b>','<p>','<pre>','<blockquote>','<del>','<dd>','<dl>','<dt>','<em>','<h1>','<h2>','<h3>','<i>','<img>','<kbd>','<li>','<ol>','<s>','<sup>','<sub>','<strong>','<ul>','<br>','<hr>']
    replacements=get_replacements(html_tags_open)
    
    for patter_to_replace, replacement in replacements:
        item = re.sub(patter_to_replace, replacement, item,flags=re.DOTALL)
    item=item.replace('$','').replace('#','').replace('=','')
    return item

In [ ]:

spark=sparknlp.start(gpu=True)

In [ ]:


user_text='i dunno'
cleaned_text=cleaning_response(user_text)
user_in={'question':cleaned_text}
schema = t.StructType([t.StructField('question', t.StringType())])

sqlContext = SQLContext(spark)
new_df=sqlContext.createDataFrame([user_in],schema)
model=PipelineModel.load('lr')
pred=model.transform(new_df).select("prediction").toPandas().values[0][0]
out={'prediction':pred}

In [ ]:



app = FastAPI()



@app.post("/predict")
async def synonyms__select(request: Request):
  data= await request.json()
  user_input=data['data']
  cleaned_text=cleaning_response(user_input)
  user_in={'question':cleaned_text}
  schema = t.StructType([t.StructField('question', t.StringType())])

  sqlContext = SQLContext(spark)
  new_df=sqlContext.createDataFrame([user_in],schema)
  model=PipelineModel.load('lr')
  pred=model.transform(new_df).select("prediction").toPandas().values[0][0]
  out={'prediction':str(pred)}

  return out

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

t=2021-04-29T19:45:17+0000 lvl=warn msg="can't bind default web address, trying alternatives" obj=web addr=127.0.0.1:4040


Public URL: http://59318a1b9d27.ngrok.io


INFO:     Started server process [59]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     35.230.116.223:0 - "POST /predict HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [59]
